In [2]:
import numpy as np
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
import os
import subprocess
import xarray as xr

import troute.nhd_io as nhd_io
import troute.nhd_network as nhd_network
import troute.nhd_network_utilities_v02 as nnu
import troute.routing.diffusive_utils as diff_utils
import troute.routing.diffusive_utils_mainstem as diff_utils_mainstem

# load MC simulation results, which will be used for initial and boundary conditions
#chrtout_folder = pathlib.Path("/glade/work/adamw/forcing/florence_event")
chrtout_folder = pathlib.Path("/home/APD/inland_hydraulics/florence_testcase/glade/work/jamesmcc/domains/private/florence_v21/FORCING_AnA_channel-only")
chrtout_files = sorted(chrtout_folder.glob("*CHRTOUT_DOMAIN1"))

with xr.open_mfdataset(
    chrtout_files, 
    combine="nested",
    concat_dim="time"
) as ds:
    
    #  lateral inflows (WRF - generated)
    q_lateral = pd.DataFrame(
        ds['qBucket'].values.T + ds['qSfcLatRunoff'].values.T,
        index=ds['feature_id'].values,
        columns=ds.time.values,
    )
    
    # wrf simulated flow (WRF - generated)
    q_wrf = pd.DataFrame(
        ds['streamflow'].values.T,
        index=ds['feature_id'].values,
        columns=ds.time.values,
    )
    
# Hand-selected segments along the Cape Fear River between gages at Tar Heel and Kelly 
#diffusive_domain = [8831712,8831726,8831730,8831734,8831736,8831748,8831754,8831752,8831780,8831794,8831792,8831798,8831808,8831810,8831806,
#                    8831818,8831820,8831826,8831830,8831832,8831840,8831846,8831848,8831852,8831858,8834892,8834894,8834898,8834904,8834908,
#                    8834912,8834918,8834924]

diffusive_domain = [8831712,8831726,8831730] # only headsegments of reaches on a mainstem.

#args = nhd_io.read_custom_input_new("/glade/u/home/adamw/projects/t-route/test/jobs/V03/Cape_Fear_test_MC_V3.yaml")
args = nhd_io.read_custom_input_new("/home/dongha.kim/github/t-route/test/input/yaml/Cape_Fear_test_MC_V3.yaml")
supernetwork_parameters = args[1]

connections, param_df, _, _ = nnu.build_connections(
        supernetwork_parameters
    )

rconn = nhd_network.reverse_network(connections)

# identify the tributary segments to the diffusive domain
trib_segs = []
for s in diffusive_domain:
    us_list = rconn[s]
    for u in us_list:
        if u not in diffusive_domain:
            trib_segs.append(u)

connections_trim = {k: connections[k] for k in (diffusive_domain + trib_segs)}
connections_trim[8831730]=[]  ## temporary fix to make proper TW link.
network_break_segments = set()
independent_networks, reaches_bytw, rconn = nnu.organize_independent_networks(
        connections_trim,
        network_break_segments,
    )

tw = list(nhd_network.headwaters(rconn))[0]
reach_list=reaches_bytw[tw]
rconn=independent_networks[tw]
connections_trim= nhd_network.reverse_network(rconn)
mainstem_headseg_list= diffusive_domain


diffusive_trib_domain=list(set(diffusive_domain)|set(trib_segs))
# mask channel geometry data
param_df_sub= param_df.loc[
                        diffusive_trib_domain,
                        ["bw", "tw", "twcc", "dx", "n", "ncc", "cs", "s0", "alt"],
                    ].sort_index()
geo_cols = param_df_sub.columns.values
geo_index = param_df_sub.index.values
geo_data = param_df_sub.values

# build initial conditions array
q0 = q_wrf.loc[geo_index].iloc[:,0].to_numpy()
q0_2 = [[val] for val in q0]
initial_conditions = np.asarray(q0_2)

# lateral inflow data
qlat_data = q_lateral.loc[geo_index].to_numpy()
# MC data
q_wrf_data= q_wrf.loc[geo_index].to_numpy()

import yaml
custom_input_file='../../../../test/input/yaml/Florence_Benchmark_diff.yaml'
with open(custom_input_file) as custom_file:
    data = yaml.load(custom_file, Loader=yaml.SafeLoader)
diffusive_parameters = data.get("diffusive_parameters", {})  
upstream_results={}
lake_segs=[]
waterbodies_df_sub = pd.DataFrame()
waterbodies_df_sub.values
lake_segs=np.asarray(lake_segs)
wbody_params=np.array(waterbodies_df_sub.values)
qts_subdivisions=12
nsteps=288
dt=60 


In [3]:
diff_inputs= diff_utils_mainstem.diffusive_mainstem_input_data_v01(
    tw,
    connections_trim,
    rconn,
    reach_list,
    mainstem_headseg_list,
    q_wrf_data,    
    diffusive_parameters,
    geo_cols,
    geo_index,
    geo_data,
    qlat_data,
    initial_conditions,
    upstream_results,
    qts_subdivisions,
    nsteps,
    dt,
    lake_segs,
    wbody_params,
)



In [4]:
import cnx_mstem_2 as cnx2
cnx2.var.timestep_ar_g= diff_inputs["timestep_ar_g"]
cnx2.var.nts_ql_g= diff_inputs["nts_ql_g"] 
cnx2.var.nts_ub_g= diff_inputs["nts_ub_g"]
cnx2.var.nts_db_g= diff_inputs["nts_db_g"] 

cnx2.var.nts_qtrib_g= diff_inputs["nts_qtrib_g"]   

cnx2.var.z_ar_g=  diff_inputs["z_ar_g"] 
cnx2.var.bo_ar_g=  diff_inputs["bo_ar_g"] 
cnx2.var.traps_ar_g= diff_inputs["traps_ar_g"] 
cnx2.var.tw_ar_g=  diff_inputs["tw_ar_g"] 
cnx2.var.twcc_ar_g= diff_inputs["twcc_ar_g"] 
cnx2.var.mann_ar_g= diff_inputs["mann_ar_g"] 
cnx2.var.manncc_ar_g= diff_inputs["manncc_ar_g"] 
cnx2.var.so_ar_g= diff_inputs["so_ar_g"] 
cnx2.var.dx_ar_g= diff_inputs["dx_ar_g"]
cnx2.var.frnw_col= diff_inputs["frnw_col"] 
cnx2.var.dfrnw_g= diff_inputs["frnw_g"] 
cnx2.var.qlat_g= diff_inputs["qlat_g"] 
cnx2.var.ubcd_g= diff_inputs["ubcd_g"] 
cnx2.var.dbcd_g= diff_inputs["dbcd_g"]   
cnx2.var.qtrib_g= diff_inputs["qtrib_g"]     
cnx2.var.paradim= diff_inputs["paradim"] 
cnx2.var.para_ar_g= diff_inputs["para_ar_g"]   
cnx2.var.iniq= diff_inputs["iniq"] 

ntss_ev_g= diff_inputs["ntss_ev_g"]
mxncomp_g= diff_inputs["mxncomp_g"] 
nrch_g= diff_inputs["nrch_g"] 

q_ev_g=np.zeros((ntss_ev_g, mxncomp_g, nrch_g))
elv_ev_g= np.zeros((ntss_ev_g, mxncomp_g, nrch_g))
#import pdb; pdb.set_trace()
q_ev_g, elv_ev_g = cnx2.diffusive.diffnw(ntss_ev_g, mxncomp_g, nrch_g)

In [6]:
q_ev_g
elv_ev_g

array([[[11.89026226, 11.89026226, 11.89026226, ..., 11.89001849,
         10.14018532, 10.14018532],
        [11.89026226, 11.89026226, 11.89026226, ..., 10.14001849,
         10.14018532, 10.10094544]],

       [[11.89523866, 11.89523866, 11.89523866, ..., 11.89036326,
         10.14369991, 10.14369991],
        [11.89523866, 11.89523866, 11.89036326, ..., 10.14369991,
         10.14369991, 10.10446002]],

       [[11.89523866, 11.89523866, 11.89523866, ..., 11.89036326,
         10.14369992, 10.14369992],
        [11.89523866, 11.89523866, 11.89036326, ..., 10.14369992,
         10.14369992, 10.10446004]],

       ...,

       [[11.89545908, 11.89545908, 11.89545908, ..., 11.89040137,
         10.14451359, 10.14451359],
        [11.89545908, 11.89545908, 11.89040137, ..., 10.14451359,
         10.14451359, 10.10531952]],

       [[11.89545913, 11.89545913, 11.89545913, ..., 11.89040138,
         10.14451456, 10.14451456],
        [11.89545913, 11.89545913, 11.89040138, ..., 10.14451

In [ ]:
'''
import cnx_mstem

timestep_ar_g= diff_inputs["timestep_ar_g"]
nts_ql_g= diff_inputs["nts_ql_g"] 
nts_ub_g= diff_inputs["nts_ub_g"]
nts_db_g= diff_inputs["nts_db_g"] 
ntss_ev_g= diff_inputs["ntss_ev_g"]
nts_qtrib_g= diff_inputs["nts_qtrib_g"]   
mxncomp_g= diff_inputs["mxncomp_g"] 
nrch_g= diff_inputs["nrch_g"] 
z_ar_g=  diff_inputs["z_ar_g"] 
bo_ar_g=  diff_inputs["bo_ar_g"] 
traps_ar_g= diff_inputs["traps_ar_g"] 
tw_ar_g=  diff_inputs["tw_ar_g"] 
twcc_ar_g= diff_inputs["twcc_ar_g"] 
mann_ar_g= diff_inputs["mann_ar_g"] 
manncc_ar_g= diff_inputs["manncc_ar_g"] 
so_ar_g= diff_inputs["so_ar_g"] 
dx_ar_g= diff_inputs["dx_ar_g"]
frnw_col= diff_inputs["frnw_col"] 
dfrnw_g= diff_inputs["frnw_g"] 
qlat_g= diff_inputs["qlat_g"] 
ubcd_g= diff_inputs["ubcd_g"] 
dbcd_g= diff_inputs["dbcd_g"]   
qtrib_g= diff_inputs["qtrib_g"]     
paradim= diff_inputs["paradim"] 
para_ar_g= diff_inputs["para_ar_g"]   
iniq= diff_inputs["iniq"] 

q_ev_g=np.zeros((ntss_ev_g, mxncomp_g, nrch_g))
elv_ev_g= np.zeros((ntss_ev_g, mxncomp_g, nrch_g))
#import pdb; pdb.set_trace()

q_ev_g, elv_ev_g = cnx_mstem.diffusive.diffnw(timestep_ar_g, 
                                            nts_ql_g, 
                                            nts_ub_g,                                           
                                            nts_db_g, 
                                            ntss_ev_g, 
                                            nts_qtrib_g, 
                                            mxncomp_g, 
                                            nrch_g, 
                                            z_ar_g, 
                                            bo_ar_g, 
                                            traps_ar_g, 
                                            tw_ar_g, 
                                            twcc_ar_g, 
                                            mann_ar_g, 
                                            manncc_ar_g, 
                                            so_ar_g, 
                                            dx_ar_g, 
                                            iniq, 
                                            frnw_col, 
                                            dfrnw_g, 
                                            qlat_g, 
                                            ubcd_g, 
                                            dbcd_g, 
                                            qtrib_g, 
                                            paradim, 
                                            para_ar_g)
'''
